In [1]:
import numpy as np
import torch
import torch.nn as nn
import random
import time
import copy
from torch.nn import functional as F
from torch.autograd import Variable
from torch import optim
from rouge import Rouge
from data import *
from utils import *

## Tokens
e.g.
```
[["Musicians to tackle US red tape Musicians ' groups are to tackle US visa regulations which are blamed for hindering",
  "Nigel McCune from the Musicians ' Union said British musicians"],
 ["U2 's desire to be number one U2 , who have won three prestigious Grammy Awards for their hit Vertigo",
  'But they still want more.They have to want to be'],
 ["Rocker Doherty in on-stage fight Rock singer Pete Doherty has been involved in a fight with his band 's guitarist",
  'Babyshambles , which he formed after his acrimonious departure from']]
```

In [2]:
train, dev = load_datasets('./Datasets/BBC_News_100_50.pkl', './Datasets/BBC_News_100_50.pkl')

In [3]:
train[0]

['Byrds producer Melcher dies at 62 Record producer Terry Melcher , who was behind hits by the Byrds , Ry Cooder and the Beach Boys , has died aged 62 . The son of actress Doris Day , he helped write Kokomo for the Beach Boys , which was used in the movie Cocktail , earning a 1988 Golden Globe nomination . He also produced Mr Tambourine Man for the Byrds , as well as other his such as Turn , Turn Turn . Melcher died on Friday night at his home in Beverly Hills , California , after a',
 "Record producer Terry Melcher , who was behind hits by the Byrds , Ry Cooder and the Beach Boys , has died aged 62.Rumours circulated that Melcher - who knew Manson - was the killer 's real target , because he had turned him down for a record contract.Melcher also"]

In [4]:
# train = train[0:10]
# dev = dev[0:1]

## Tokens Index
e.g.
```
[Musicians to tackle US red tape Musicians ' groups are to tackle US visa regulations which are blamed for hindering => Nigel McCune from the Musicians ' Union said British musicians
    indexed as: [2, 3, 4, 5, 6, 7, 2, 8, 9, 10, 3, 4, 5, 11, 12, 13, 10, 14, 15, 16] => [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 2],
 U2 's desire to be number one U2 , who have won three prestigious Grammy Awards for their hit Vertigo => But they still want more.They have to want to be
    indexed as: [17, 18, 19, 3, 20, 21, 22, 17, 23, 24, 25, 26, 27, 28, 29, 30, 15, 31, 32, 33] => [13, 14, 15, 16, 17, 18, 19, 16, 19, 20, 2],
 Rocker Doherty in on-stage fight Rock singer Pete Doherty has been involved in a fight with his band 's guitarist => Babyshambles , which he formed after his acrimonious departure from
    indexed as: [34, 35, 36, 37, 38, 39, 40, 41, 35, 42, 43, 44, 36, 45, 38, 46, 47, 48, 18, 49] => [21, 22, 23, 24, 25, 26, 27, 28, 29, 5, 2]]
```

In [5]:
train_data_indexed, dev_data_indexed, vocab_indexer = index_datasets(train, dev)

## Padding
- Pad the train/dev input vectors to the max length of the train/dev input documents.
- Pad the train/dev output vectors to the max length of the train/dev output summerization.

![](https://i.imgur.com/gGlkEEF.png)

In [6]:
def make_padded_input_tensor(exs, vocab_indexer, max_len):
    return np.array([[ex.x_indexed[i] if i < len(ex.x_indexed) else vocab_indexer.index_of(PAD_SYMBOL)
                        for i in range(0, max_len)] for ex in exs])

In [7]:
def make_padded_output_tensor(exs, vocab_indexer, max_len):
    return np.array([[ex.y_indexed[i] if i < len(ex.y_indexed) else vocab_indexer.index_of(PAD_SYMBOL)
                        for i in range(0, max_len)] for ex in exs])

## Batch

In [8]:
def batch_data(input_array, batch_size=2, cuda=False):
    input_batches = []
    batch_num = (int)(input_array.shape[0] / batch_size)
    start = 0
    for i in range(batch_num):
        batch = torch.from_numpy(input_array[start:start+batch_size, :])
        if cuda:
            batch = batch.cuda()
        input_batches.append(batch)
        start += batch_size
    if start != input_array.shape[0]:
        batch = torch.from_numpy(input_array[start:, :])
        if cuda:
            batch = batch.cuda()
        input_batches.append(batch)
    return input_batches

## Embedding

In [9]:
class EmbeddingLayer(nn.Module):
    # Parameters: dimension of the word embeddings, number of words, and the dropout rate to apply
    # (0.2 is often a reasonable value)
    def __init__(self, input_dim, full_dict_size, embedding_dropout_rate):
        super(EmbeddingLayer, self).__init__()
        self.dropout = nn.Dropout(embedding_dropout_rate)
        self.word_embedding = nn.Embedding(full_dict_size, input_dim)

    # Takes either a non-batched input [sent len x input_dim] or a batched input
    # [batch size x sent len x input dim]
    def forward(self, input):
        embedded_words = self.word_embedding(input)
        final_embeddings = self.dropout(embedded_words)
        return final_embeddings

## Encoder

In [10]:
# One-layer RNN encoder for batched inputs -- handles multiple sentences at once. You're free to call it with a
# leading dimension of 1 (batch size 1) but it does expect this dimension.
class RNNEncoder(nn.Module):
    # Parameters: input size (should match embedding layer), hidden size for the LSTM, dropout rate for the RNN,
    # and a boolean flag for whether or not we're using a bidirectional encoder
    def __init__(self, input_size, hidden_size, dropout, bidirect, CUDA=False):
        super(RNNEncoder, self).__init__()
        self.CUDA = CUDA
        self.bidirect = bidirect
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.reduce_h_W = nn.Linear(hidden_size * 2, hidden_size, bias=True)
        self.reduce_c_W = nn.Linear(hidden_size * 2, hidden_size, bias=True)
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers=1, batch_first=True,
                               dropout=dropout, bidirectional=self.bidirect)
        self.init_weight()

    # Initializes weight matrices using Xavier initialization
    def init_weight(self):
        nn.init.xavier_uniform_(self.rnn.weight_hh_l0, gain=1)
        nn.init.xavier_uniform_(self.rnn.weight_ih_l0, gain=1)
        if self.bidirect:
            nn.init.xavier_uniform_(self.rnn.weight_hh_l0_reverse, gain=1)
            nn.init.xavier_uniform_(self.rnn.weight_ih_l0_reverse, gain=1)
        nn.init.constant_(self.rnn.bias_hh_l0, 0)
        nn.init.constant_(self.rnn.bias_ih_l0, 0)
        if self.bidirect:
            nn.init.constant_(self.rnn.bias_hh_l0_reverse, 0)
            nn.init.constant_(self.rnn.bias_ih_l0_reverse, 0)

    def get_output_size(self):
        return self.hidden_size * 2 if self.bidirect else self.hidden_size

    def sent_lens_to_mask(self, lens, max_length):
        return torch.from_numpy(np.asarray([[1 if j < lens.data[i].item() else 0 for j in range(0, max_length)] for i in range(0, lens.shape[0])]))

    # embedded_words should be a [batch size x sent len x input dim] tensor
    # input_lens is a tensor containing the length of each input sentence
    # Returns output (each word's representation), context_mask (a mask of 0s and 1s
    # reflecting where the model's output should be considered), and h_t, a *tuple* containing
    # the final states h and c from the encoder for each sentence.
    def forward(self, embedded_words, input_lens):
        # Takes the embedded sentences, "packs" them into an efficient Pytorch-internal representation
        packed_embedding = nn.utils.rnn.pack_padded_sequence(embedded_words, input_lens, batch_first=True)

        # Runs the RNN over each sequence. Returns output at each position as well as the last vectors of the RNN
        # state for each sentence (first/last vectors for bidirectional)
        output, hn = self.rnn(packed_embedding)
        

        # Unpacks the Pytorch representation into normal tensors
        output, sent_lens = nn.utils.rnn.pad_packed_sequence(output)
#         print('kdjfksdjfs: ', output.shape)
        
        # print('input_lens:', input_lens)
        max_length = input_lens.data[0].item()
        context_mask = self.sent_lens_to_mask(sent_lens, max_length)
        if self.CUDA:
            context_mask = context_mask.cuda()

        # Grabs the encoded representations out of hn, which is a weird tuple thing.
        # Note: if you want multiple LSTM layers, you'll need to change this to consult the penultimate layer
        # or gather representations from all layers.
        if self.bidirect:
            h, c = hn[0], hn[1]          # [2, 20, 200]
            # print('encoder hidden:----- ', h.shape)
            # print('encoder cell:----- ', c.shape)
            # Grab the representations from forward and backward LSTMs
            h_, c_ = torch.cat((h[0], h[1]), dim=1), torch.cat((c[0], c[1]), dim=1)      # [20, 400]
            # print('kdjfksdddddddddddjfs: ',h_.shape)
            # Reduce them by multiplying by a weight matrix so that the hidden size sent to the decoder is the same
            # as the hidden size in the encoder
            new_h = self.reduce_h_W(h_)
            new_c = self.reduce_c_W(c_)
            h_t = (new_h, new_c)
        else:
            h, c = hn[0][0], hn[1][0]
            h_t = (h, c)
        return (output, context_mask, h_t)

## Attention-based Decoder

In [11]:
class AttnRNNDecoderBahdanau(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout, bidirect):
        super(AttnRNNDecoderBahdanau, self).__init__()

        self.input_size = input_size
#         self.sent_lens = sent_lens
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout = dropout
        self.bidirect = bidirect
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers=1, 
                                dropout=dropout, bidirectional=bidirect)
        
        
        self.context = nn.Linear(hidden_size * 2 + input_size, input_size)
        self.W_h = nn.Linear(2*hidden_size, 2*hidden_size)
        self.W_c = nn.Linear(1, hidden_size * 2, bias=False)
        self.W_s = nn.Linear(hidden_size * 2, hidden_size * 2)
        self.v = nn.Linear(hidden_size * 2, 1, bias=False)
        self.V = nn.Linear(hidden_size * 2 + hidden_size, hidden_size)
        self.V_p = nn.Linear(hidden_size, output_size)
        self.P_gen_layer = nn.Linear(hidden_size * 4 + input_size, 1)

        self.init_weight()
        
    # Initializes weight matrices using Xavier initialization
    def init_weight(self):
        nn.init.xavier_uniform_(self.rnn.weight_hh_l0, gain=1)
        nn.init.xavier_uniform_(self.rnn.weight_ih_l0, gain=1)
        if self.bidirect:
            nn.init.xavier_uniform_(self.rnn.weight_hh_l0_reverse, gain=1)
            nn.init.xavier_uniform_(self.rnn.weight_ih_l0_reverse, gain=1)
        nn.init.constant_(self.rnn.bias_hh_l0, 0)
        nn.init.constant_(self.rnn.bias_ih_l0, 0)
        if self.bidirect:
            nn.init.constant_(self.rnn.bias_hh_l0_reverse, 0)
            nn.init.constant_(self.rnn.bias_ih_l0_reverse, 0)

    def forward(self, embedded_words, dec_hidden, enc_output, context_mask, pre_cont, coverage):
        sent_lens = enc_output.shape[0]
        enc_feature = enc_output.view(-1, 2*hidden_size)                # batch_size*sent_lens, 2*hidden_size
        rnn_input = self.context(torch.cat((pre_cont, embedded_words), 1))   # batch_size, input_size
        dec_output, hn = self.rnn(rnn_input.unsqueeze(1).transpose(0, 1), dec_hidden)   # 1, batch_size, hidden_size
        dec_output = dec_output.transpose(0, 1)                             # batch_size, 1, hidden_size
        h_dec, c_dec = hn
        s_t_hat = torch.cat((h_dec.view(-1, hidden_size),
                             c_dec.view(-1, hidden_size)), 1)           # batch_size, 2*hidden_size
        del h_dec, c_dec
        
        # Attention Distribution
        dec_state = self.W_s(s_t_hat)                              # batch_size, 2*hidden_size
        dec_state_expanded = dec_state.unsqueeze(1).expand(dec_state.shape[0], sent_lens, dec_state.shape[1]).contiguous() # batch_size, sent_lens, 2*hidden_size
        dec_state_expanded = dec_state_expanded.view(-1, dec_state.shape[1])  # batch_size*sent_lens, 2*hidden_size
        del dec_state
        
#         print(enc_feature.shape)
#         print(dec_state_expanded.shape)
#         print(coverage.shape)
        e = self.v(torch.tanh(self.W_h(enc_feature) + dec_state_expanded + self.W_c(coverage.view(-1, 1)))).view(-1, sent_lens)  # batch_size, sent_lens
#         att_feature = enc_feature + dec_state_expanded   # batch_size*sent_lens, 2*hidden_size
#         # Coverage
#         coverage_feature = self.W_c(coverage.view(-1, 1))  # batch_size*sent_lens, 2*hidden_size
#         att_feature = att_feature + coverage_feature   # batch_size*sent_lens, 2*hidden_size
#         e = torch.tanh(att_feature)       # batch_size*sent_lens, 2*hidden_size
#         attn_scores = self.v(e).view(-1, self.sent_lens)      # batch_size, sent_lens
        del enc_feature
        attn_distrib_ = F.softmax(e, dim=1)*context_mask.float()   # batch_size, sent_lens
        del e
        norm_factor = attn_distrib_.sum(1, keepdim=True)
        attn_distrib = attn_distrib_ / norm_factor
        del attn_distrib_, norm_factor
        coverage = coverage + attn_distrib                  # batch_size, sent_lens
        attn_distrib = attn_distrib.unsqueeze(1)            # batch_size, 1, sent_lens
        
        # Context Vector
        cont_vec = torch.bmm(attn_distrib, enc_outputs.transpose(0, 1))  # batch_size, 1, 2*hidden_size
        concat_input = torch.cat((dec_output, cont_vec), dim=-1)           # batch_size, 1, enc_hidden_size * num_directions + dec_hidden_size

        # Vocabulary Distribution
        vocab_distrib = torch.softmax(self.V_p(self.V(concat_input)), dim=-1).squeeze(1)  # batch_size, output_size

        # Pointer Generator
        P_gen_input = torch.cat((cont_vec.squeeze(1), s_t_hat, rnn_input), dim=1)  # batch_size, (2*2*hidden_size + input_size)
        P_gen = self.P_gen_layer(P_gen_input)
        P_gen = torch.sigmoid(P_gen)       # batch_size, 1
        
        return (P_gen, vocab_distrib, hn, attn_distrib.squeeze(1), cont_vec.squeeze(1), coverage)
        

In [12]:
class AttnRNNDecoder(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout, bidirect):
        super(AttnRNNDecoder, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout = dropout
        self.bidirect = bidirect
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers=1, 
                                dropout=dropout, bidirectional=bidirect)
        self.out = nn.Linear(hidden_size, output_size)
        self.concat = nn.Linear(hidden_size * 2 + hidden_size, hidden_size)
        self.linear = nn.Linear(hidden_size * 2, hidden_size)
        
        self.W_h = nn.Linear(hidden_size * 2, 1)
        self.W_s = nn.Linear(hidden_size, 1)
        self.W_x = nn.Linear(input_size, 1)
        self.init_weight()
        
    # Initializes weight matrices using Xavier initialization
    def init_weight(self):
        nn.init.xavier_uniform_(self.rnn.weight_hh_l0, gain=1)
        nn.init.xavier_uniform_(self.rnn.weight_ih_l0, gain=1)
        if self.bidirect:
            nn.init.xavier_uniform_(self.rnn.weight_hh_l0_reverse, gain=1)
            nn.init.xavier_uniform_(self.rnn.weight_ih_l0_reverse, gain=1)
        nn.init.constant_(self.rnn.bias_hh_l0, 0)
        nn.init.constant_(self.rnn.bias_ih_l0, 0)
        if self.bidirect:
            nn.init.constant_(self.rnn.bias_hh_l0_reverse, 0)
            nn.init.constant_(self.rnn.bias_ih_l0_reverse, 0)

    # enc_output: batch_size, sent_lens, 2*hidden_size
    def forward(self, embedded_words, dec_hidden, enc_outputs, context_mask):
        embedded_words = embedded_words.view(1, embedded_words.size(0), embedded_words.size(1))   # 1, batch_size, input_size
        context_mask = context_mask.type(torch.uint8).unsqueeze(1)      # batch_size, 1, sent_lens
  
        rnn_output, hn = self.rnn(embedded_words, dec_hidden)           # 1, batch_size, hidden_size
        rnn_output = rnn_output.transpose(0, 1)                         # batch_size, 1, hidden_size        
        
        attn_scores = rnn_output.bmm(self.linear(enc_outputs).transpose(0, 1).transpose(1, 2))      # batch_size, 1, sent_lens
        attn_scores.data.masked_fill(context_mask == 0, float('inf'))      # batch_size, 1, sent_lens
#         print(attn_scores.shape)
        
        # Attention Distribution
        attn_weights = F.softmax(attn_scores.squeeze(1), dim=1).unsqueeze(1)        # batch_size, 1, sent_lens
        # Context Vector
        context = attn_weights.bmm(enc_outputs.transpose(0, 1))                     # batch_size, 1, hidden_size * num_directions
        concat_input = torch.cat((context, rnn_output), dim=-1)     # batch_size, 1, enc_hidden_size * num_directions + dec_hidden_size
        concat_output = torch.tanh(self.concat(concat_input))                       # batch_size, 1, dec_hidden_size
        # Vocabulary Distribution
        output = self.out(concat_output).squeeze(1)                                 # batch_size, output_size
        # Pointer-Generator
        p_gen = torch.sigmoid(self.W_h(context) + self.W_s(rnn_output) + self.W_x(embedded_words.transpose(0, 1))).squeeze(1)  # batch_size, 1
#         print(p_gen.shape)
        
        return (p_gen, output, hn, attn_weights.squeeze(1))

## Encoder to Decoder

In [13]:
def encode_input_for_decoder(x_tensor, inp_lens_tensor, model_input_emb, model_enc):
    input_emb = model_input_emb.forward(x_tensor)
    (enc_output_each_word, enc_context_mask, enc_final_states) = model_enc.forward(input_emb, inp_lens_tensor)
    enc_final_states_reshaped = (enc_final_states[0].unsqueeze(0), enc_final_states[1].unsqueeze(0))
    return (enc_output_each_word, enc_context_mask, enc_final_states_reshaped)

## Loss Function

In [14]:
# Implementation of loss function: masked cross entropy
# Reference to https://github.com/spro/practical-pytorch, make some modifications
def masked_cross_entropy(logits, target, length, context_mask):
    logits_flat = logits.view(-1, logits.size(-1))                  # batch_size * sent_len, vocab_size
    log_probs_flat = F.log_softmax(logits_flat, dim=-1)             # batch_size * sent_len, vocab_size
    target_flat = target.view(-1, 1)                                # batch * sent_len, 1
    losses_flat = -torch.gather(log_probs_flat, dim=1, index=target_flat)    # batch * max_len, 1
    losses = losses_flat.view(*target.size())     # batch, max_len
    losses = losses * context_mask.float()
    loss = losses.sum() / length.float().sum()
    return loss

In [15]:
def cal_step_loss(final_distrib, attn_distrib, coverage, step_Y_tensor, step_context_mask):
    step_distrib = torch.gather(final_distrib, dim=1, index=step_Y_tensor).squeeze(1)
    step_coverage_loss = torch.sum(torch.min(attn_distrib, coverage), 1)
    step_loss = -torch.log(step_distrib + eps) + cov_loss_wt*step_coverage_loss
    step_loss = step_loss * step_context_mask
    return step_loss

## Training Step

**Training Copy**

In [16]:
def pointer_generate_train(p_gen, dec_output, dec_attn, X_tensors, CUDA):
#     final_distrib = torch.zeros(dec_output.shape, dtype=torch.float)     # batch_size, vocab_size
    final_distrib = p_gen * dec_output
    dec_attn_padding = torch.zeros(X_tensors.shape, dtype=torch.float)
    if CUDA:
        dec_attn_padding = dec_attn_padding.cuda()
    dec_attn_padding[:, 0:dec_attn.shape[1]] = dec_attn 
    final_distrib = final_distrib.scatter_add(1, X_tensors, (1-p_gen)*dec_attn_padding)
    return final_distrib

In [17]:
BATCH_SIZE = 8
lr = 0.0005
input_dim = 100
output_dim = 100
hidden_size = 256
emb_dropout = 0.2
rnn_dropout = 0.2
bidirectional = True
num_epochs = 500
teacher_forcing_ratio = 1
cov_loss_wt = 1.0
eps = 1e-12
CUDA = True
pad_idx = vocab_indexer.index_of(PAD_SYMBOL)
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

**Create indexed input/output for training**
- X_tensors_batch/Y_tensors_batch, list[array: [batch_size, sent_len], batch_num]
- inp_lens_batch/oup_lens_batch, list[array: [batch_size,], batch_num]

In [18]:
# Create indexed input/output for training
train_data_indexed.sort(key=lambda ex: len(ex.x_indexed), reverse=True)
input_train_max_len = np.max(np.asarray([len(ex.x_indexed) for ex in train_data_indexed]))
all_train_input_data = make_padded_input_tensor(train_data_indexed, vocab_indexer, input_train_max_len).astype(np.int64)

output_train_max_len = np.max(np.asarray([len(ex.y_indexed) for ex in train_data_indexed]))
all_train_output_data = make_padded_output_tensor(train_data_indexed, vocab_indexer, output_train_max_len).astype(np.int64)

X_tensors_batch = batch_data(all_train_input_data, BATCH_SIZE, cuda=CUDA)   # batch_num, batch_size, sent_len
Y_tensors_batch = batch_data(all_train_output_data, BATCH_SIZE, cuda=CUDA)  # batch_num, batch_size, sent_len
if CUDA:
    inp_lens_batch = [torch.tensor([torch.sum(X_tensor != 0) for X_tensor in X_tensors]).cuda() for X_tensors in X_tensors_batch]  # batch_num, batch_size
    oup_lens_batch = [torch.tensor([torch.sum(Y_tensor != 0) for Y_tensor in Y_tensors]).cuda() for Y_tensors in Y_tensors_batch]  # batch_num, batch_size
else:
    inp_lens_batch = [torch.tensor([torch.sum(X_tensor != 0) for X_tensor in X_tensors]) for X_tensors in X_tensors_batch]  # batch_num, batch_size
    oup_lens_batch = [torch.tensor([torch.sum(Y_tensor != 0) for Y_tensor in Y_tensors]) for Y_tensors in Y_tensors_batch]  # batch_num, batch_size

**Create model**
- model_input_emb/model_output_emb: embedding layer
- model_enc/model_dec: encoder/decoder
- optimizers: encoder/decoder

In [ ]:
# Create model
model_input_emb = EmbeddingLayer(input_dim, len(vocab_indexer), emb_dropout)
model_enc = RNNEncoder(input_dim, hidden_size, rnn_dropout, bidirectional, CUDA=CUDA)
model_output_emb = EmbeddingLayer(output_dim, len(vocab_indexer), emb_dropout)
model_dec = AttnRNNDecoderBahdanau(input_size=output_dim, hidden_size=hidden_size, output_size=len(vocab_indexer), dropout=rnn_dropout, bidirect=False)
# CUDA
if CUDA:
    model_input_emb.cuda()
    model_enc.cuda()
    model_output_emb.cuda()
    model_dec.cuda()
# model_dec = AttnRNNDecoder(input_size=output_dim, hidden_size=hidden_size, output_size=len(vocab_indexer), dropout=rnn_dropout, bidirect=False)
enc_optimizer = optim.Adam(model_enc.parameters(), lr=lr)
dec_optimizer = optim.Adam(model_dec.parameters(), lr=lr)

c:\python36_64\lib\site-packages\torch\nn\modules\rnn.py:38: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


**Train Iteration**

In [ ]:
start = time.time()
for epoch in range(0, num_epochs):
    print('--------------------- Epoch %d ---------------------'%(epoch+1))
    for X_tensors, Y_tensors, inp_lens_tensor, oup_lens_tensor in zip(X_tensors_batch, Y_tensors_batch, inp_lens_batch, oup_lens_batch):

        model_enc.train()
        model_dec.train()

        enc_optimizer.zero_grad()
        dec_optimizer.zero_grad()
        
        # Encoder
        enc_outputs, enc_context_mask, enc_hidden = encode_input_for_decoder(X_tensors, inp_lens_tensor, model_input_emb, model_enc)
        init_dec_inp = Variable(torch.LongTensor([vocab_indexer.index_of(SOS_SYMBOL)] * X_tensors.shape[0]))  
        if CUDA:
            init_dec_inp = init_dec_inp.cuda()
        dec_input = model_output_emb.forward(init_dec_inp)
        if TOPIC:
            init_dec_inp
        
        
        
        dec_hidden = enc_hidden
        cont = torch.zeros((X_tensors.shape[0], 2 * hidden_size))    # batch_size, 2*hidden_size
        coverage = torch.zeros((X_tensors.shape[0], enc_outputs.shape[0]))             # batch_size, sent_lens
        all_context_mask = torch.from_numpy(np.asarray([[1 if j < oup_lens_tensor.data[i].item() \
            else 0 for j in range(0, Y_tensors.size(1))] for i in range(0, oup_lens_tensor.shape[0])]))
        agr_loss = torch.zeros(X_tensors.shape[0])
        
        if CUDA:
            cont = cont.cuda()
            coverage = coverage.cuda()
            all_context_mask = all_context_mask.cuda()
            agr_loss = agr_loss.cuda()
        
#         # Decoder
#         all_dec_outputs = Variable(torch.zeros(output_train_max_len, X_tensors.shape[0], len(vocab_indexer)))   # sent_len, batch_size, ext_output_size
#         for idx in range(output_train_max_len):
#             p_gen, dec_output, dec_hidden, dec_attn = model_dec.forward(dec_input, dec_hidden, enc_outputs, enc_context_mask)
#             all_dec_outputs[idx] = pointer_generate_train(p_gen, dec_output, dec_attn, X_tensors)
#             max_prob_idx = torch.argmax(all_dec_outputs[idx], dim=1)
#             print('max_prob_idx:', max_prob_idx)
#             print('Y_tensors:', Y_tensors[:, idx])
# #             print(agr_loss)
# #             agr_loss = agr_loss + cal_step_loss(final_distrib, dec_attn, coverage, Y_tensors[:, idx].unsqueeze(1), all_context_mask[:, idx].float())
#             dec_input = model_output_emb.forward(Y_tensors[:, idx])
# #             coverage = next_coverage         
#         loss = masked_cross_entropy(all_dec_outputs.transpose(0, 1).contiguous(), Y_tensors, oup_lens_tensor, all_context_mask)       # batch_size, sent_len, output_size
        
        
        # Decoder2
        for idx in range(output_train_max_len):
            p_gen, dec_output, dec_hidden, dec_attn, cont, next_coverage = model_dec.forward(dec_input, dec_hidden, enc_outputs, enc_context_mask, cont, coverage)
            final_distrib = pointer_generate_train(p_gen, dec_output, dec_attn, X_tensors, CUDA)
            max_prob_idx = torch.argmax(final_distrib, dim=1)
#             print(final_distrib)
#             print('max_prob_idx:', max_prob_idx)
#             print('Y_tensors:', Y_tensors[:, idx])
#             print(agr_loss)
            agr_loss = agr_loss + cal_step_loss(final_distrib, dec_attn, coverage, Y_tensors[:, idx].unsqueeze(1), all_context_mask[:, idx].float())
            dec_input = model_output_emb.forward(Y_tensors[:, idx])
            coverage = next_coverage
        
        batch_avg_loss = agr_loss/oup_lens_tensor.float()
        loss = torch.mean(batch_avg_loss)
        
        loss.backward()

        enc_optimizer.step()
        dec_optimizer.step()

        print('loss', loss.item())
        
elapsed_time = time.time() - start
print('Time: %.2fs'%(elapsed_time))

--------------------- Epoch 1 ---------------------
loss 6.682219982147217
loss 7.096613883972168
loss 7.289017200469971
loss 6.96723747253418
loss 6.931626796722412
loss 7.120256423950195
loss 6.948846817016602
loss 7.004876136779785
loss 6.574513912200928
loss 6.941911220550537
loss 7.446579456329346
loss 6.987937927246094
loss 7.371664524078369
loss 7.43435525894165
loss 7.313004016876221
loss 6.834506988525391
loss 7.0308966636657715
loss 6.51055383682251
loss 7.370500087738037
loss 6.93660831451416
loss 6.362265110015869
loss 6.800319671630859
loss 6.689985275268555
loss 6.918300151824951
loss 7.141538143157959
loss 6.856302738189697
loss 6.781208038330078
loss 6.575254917144775
loss 6.801943778991699
loss 6.5886383056640625
loss 6.740764617919922
loss 6.209800720214844
loss 6.7847723960876465
loss 6.911956787109375
loss 6.4391865730285645
loss 7.091922760009766
loss 6.662973403930664
loss 6.254928112030029
loss 6.951873302459717
loss 6.890693664550781
loss 6.748363971710205
loss 

loss 6.260390758514404
loss 6.039236068725586
loss 6.219856262207031
loss 5.9032793045043945
loss 6.230953693389893
loss 6.319899559020996
loss 5.956056594848633
loss 6.485411643981934
loss 6.171507358551025
loss 5.820878982543945
loss 6.405047416687012
loss 6.332647323608398
loss 6.217918395996094
loss 6.540416717529297
loss 6.588804244995117
loss 6.134335517883301
loss 6.196417808532715
loss 6.239518165588379
loss 6.1439032554626465
loss 6.115933418273926
loss 5.918435096740723
loss 5.942765235900879
loss 6.086821556091309
loss 6.1803154945373535
loss 6.166346549987793
loss 6.187134265899658
loss 6.360445976257324
loss 6.438915729522705
loss 6.3634724617004395
loss 6.174997806549072
loss 6.229969501495361
loss 5.974547386169434
loss 5.883945465087891
loss 6.489138603210449
loss 5.7050275802612305
--------------------- Epoch 7 ---------------------
loss 5.894809722900391
loss 6.251573085784912
loss 6.28465461730957
loss 6.112685203552246
loss 6.0316572189331055
loss 6.224917888641357


loss 5.663633823394775
loss 5.559226036071777
loss 5.378233909606934
loss 5.827363014221191
loss 4.8823957443237305
--------------------- Epoch 12 ---------------------
loss 5.395224571228027
loss 5.688724040985107
loss 5.661533355712891
loss 5.559519290924072
loss 5.530464172363281
loss 5.685520648956299
loss 5.600069999694824
loss 5.602293968200684
loss 5.45303201675415
loss 5.569540977478027
loss 5.684627056121826
loss 5.6933183670043945
loss 5.931724548339844
loss 5.785111904144287
loss 5.722293376922607
loss 5.381943702697754
loss 5.50319242477417
loss 5.412695407867432
loss 5.624261856079102
loss 5.544377326965332
loss 5.404471397399902
loss 5.6783928871154785
loss 5.438896179199219
loss 5.6447296142578125
loss 5.797823429107666
loss 5.651298999786377
loss 5.646653175354004
loss 5.531062602996826
loss 5.651065349578857
loss 5.437340259552002
loss 5.59537410736084
loss 5.422532081604004
loss 5.60612154006958
loss 5.601322650909424
loss 5.348388671875
loss 5.889214992523193
loss 5.

loss 4.920263767242432
loss 5.168356418609619
loss 4.959718227386475
loss 4.9899492263793945
loss 4.906375885009766
loss 4.996639251708984
loss 4.86946439743042
loss 5.021608829498291
loss 4.820359706878662
loss 4.95493221282959
loss 5.084301948547363
loss 4.754262924194336
loss 5.126285552978516
loss 4.780130386352539
loss 4.684189796447754
loss 4.808468818664551
loss 4.876697540283203
loss 4.888808727264404
loss 5.071101665496826
loss 5.021739959716797
loss 4.58389139175415
loss 4.5764875411987305
loss 5.059024810791016
loss 4.799126148223877
loss 4.7855682373046875
loss 4.772384166717529
loss 4.911135673522949
loss 4.758112907409668
loss 4.584278583526611
loss 4.814074516296387
loss 4.7244873046875
loss 4.8671698570251465
loss 4.916433334350586
loss 4.7411885261535645
loss 4.774366855621338
loss 4.817635536193848
loss 4.8128228187561035
loss 4.595564842224121
loss 4.851517677307129
loss 3.6006150245666504
--------------------- Epoch 18 ---------------------
loss 4.661112308502197
lo

loss 4.011950492858887
loss 4.225798606872559
loss 4.309284687042236
loss 4.071982383728027
loss 4.080846786499023
loss 4.158544540405273
loss 4.202315330505371
loss 4.025012016296387
loss 4.206754684448242
loss 2.6888062953948975
--------------------- Epoch 23 ---------------------
loss 3.806586980819702
loss 4.033792495727539
loss 3.953136920928955
loss 4.041162490844727
loss 4.002616882324219
loss 4.10928201675415
loss 4.140981197357178
loss 4.157440185546875
loss 4.159093856811523
loss 4.170629024505615
loss 3.9149816036224365
loss 4.3233323097229
loss 4.168878078460693
loss 4.109511852264404
loss 3.790921926498413
loss 3.7835798263549805
loss 3.7018251419067383
loss 3.9634125232696533
loss 3.8100860118865967
loss 3.6996474266052246
loss 3.9601311683654785
loss 4.1524553298950195
loss 3.978564500808716
loss 3.9883439540863037
loss 4.354497909545898
loss 4.1356024742126465
loss 4.034270763397217
loss 3.9664723873138428
loss 4.211947917938232
loss 4.203221321105957
loss 4.05123758316

...

loss 0.44757911562919617
loss 0.41896316409111023
loss 0.4092648923397064
loss 0.35673367977142334
loss 0.4784468710422516
loss 0.43969687819480896
loss 0.5014187097549438
loss 0.4225154519081116
loss 0.376264363527298
loss 0.3603043854236603
loss 0.5562562942504883
loss 0.343655526638031
loss 0.4241122603416443
loss 0.40798550844192505
loss 0.33915582299232483
loss 0.36804014444351196
loss 0.27574723958969116
--------------------- Epoch 180 ---------------------
loss 0.5414193868637085
loss 0.6627988219261169
loss 0.3782949149608612
loss 0.32825160026550293
loss 0.3961640000343323
loss 0.434635728597641
loss 0.31571251153945923
loss 0.3760494887828827
loss 0.4105933904647827
loss 0.4100518822669983
loss 0.37493348121643066
loss 0.3023000657558441
loss 0.44814205169677734
loss 0.4242839217185974
loss 0.34017103910446167
loss 0.3311592638492584
loss 0.3298012614250183
loss 0.4741256535053253
loss 0.40464383363723755
loss 0.38754457235336304
loss 0.367891788482666
loss 0.3577703833580017

loss 0.35298585891723633
loss 0.2330114096403122
loss 0.32278308272361755
loss 0.27533772587776184
loss 0.24566033482551575
loss 0.3623916506767273
loss 0.11196532845497131
--------------------- Epoch 185 ---------------------
loss 0.3376689553260803
loss 0.5497244000434875
loss 0.303877592086792
loss 0.29495900869369507
loss 0.22577090561389923
loss 0.32315877079963684
loss 0.2561693787574768
loss 0.2824965715408325
loss 0.23808154463768005
loss 0.24601252377033234
loss 0.356723427772522
loss 0.35124701261520386
loss 0.3238462805747986
loss 0.3492809236049652
loss 0.2623760998249054
loss 0.27447834610939026
loss 0.3105948567390442
loss 0.2825635075569153
loss 0.32647544145584106
loss 0.334696888923645
loss 0.29319262504577637
loss 0.3366376757621765
loss 0.26727718114852905
loss 0.30219942331314087
loss 0.28381338715553284
loss 0.24585272371768951
loss 0.2961648404598236
loss 0.4219072461128235
loss 0.32055357098579407
loss 0.24657416343688965
loss 0.31075459718704224
loss 0.352739453

loss 0.28089287877082825
loss 0.2647789716720581
loss 0.22180038690567017
loss 0.2367183268070221
loss 0.28696534037590027
loss 0.2737051844596863
loss 0.3668941259384155
loss 0.24013596773147583
loss 0.27913007140159607
loss 0.23613768815994263
loss 0.335144579410553
loss 0.28783121705055237
loss 0.24721869826316833
loss 0.2577897608280182
loss 0.2768748998641968
loss 0.3146592080593109
loss 0.2546342611312866
loss 0.24032188951969147
loss 0.31872451305389404
loss 0.3598247766494751
loss 0.2998592257499695
loss 0.2712849974632263
loss 0.24139247834682465
loss 0.2736872434616089
loss 0.34960120916366577
loss 0.32916632294654846
loss 0.34357088804244995
loss 0.32783159613609314
loss 0.2632695734500885
loss 0.27975520491600037
loss 0.30359455943107605
loss 0.2476455718278885
loss 0.30519339442253113
loss 0.270489364862442
loss 0.2714800238609314
loss 0.2828855812549591
loss 0.28194254636764526
loss 0.2680009603500366
loss 0.31076616048812866
loss 0.31285104155540466
loss 0.33680999279022

loss 0.2625613212585449
loss 0.2976694703102112
loss 0.26890695095062256
loss 0.25721216201782227
loss 0.2830163240432739
loss 0.25534096360206604
loss 0.2886734902858734
loss 0.1961883157491684
loss 0.28583449125289917
loss 0.2778281569480896
loss 0.24455800652503967
loss 0.2856454849243164
loss 0.2722930610179901
loss 0.2574165165424347
loss 0.2500511407852173
loss 0.20395199954509735
loss 0.24553315341472626
loss 0.36373311281204224
loss 0.2660362124443054
loss 0.2413739264011383
loss 0.2988205552101135
loss 0.28100916743278503
loss 0.2519508898258209
loss 0.21296226978302002
loss 0.2799088656902313
loss 0.2491341233253479
loss 0.2711094617843628
loss 0.251290500164032
loss 0.25033077597618103
loss 0.29723796248435974
loss 0.2849252223968506
loss 0.2406284660100937
loss 0.25532203912734985
loss 0.24296322464942932
loss 0.27073267102241516
loss 0.2276005893945694
loss 0.345211923122406
loss 0.30983766913414
loss 0.20605915784835815
loss 0.33377474546432495
loss 0.2657943069934845
los

loss 0.2854786217212677
loss 0.25735270977020264
loss 0.24352619051933289
loss 0.22957095503807068
loss 0.2729280889034271
loss 0.21328508853912354
loss 0.3049382269382477
loss 0.28084754943847656
loss 0.24379555881023407
loss 0.20401175320148468
loss 0.2330690622329712
loss 0.24366790056228638
loss 0.26052457094192505
loss 0.21008838713169098
loss 0.2748032212257385
loss 0.2894152104854584
loss 0.3148672580718994
loss 0.2787923514842987
loss 0.2649741768836975
loss 0.26757150888442993
loss 0.27886196970939636
loss 0.278413325548172
loss 0.3081001341342926
loss 0.28751111030578613
loss 0.24614350497722626
loss 0.2326594442129135
loss 0.2653445899486542
loss 0.2608376443386078
loss 0.24052771925926208
loss 0.3203868567943573
loss 0.30444949865341187
loss 0.2663339078426361
loss 0.24511095881462097
loss 0.23537932336330414
loss 0.30617621541023254
loss 0.30178219079971313
loss 0.24452939629554749
loss 0.2328839898109436
loss 0.2492828667163849
loss 0.24118930101394653
loss 0.265377998352

In [42]:
with open('./Models/train_400_120_enc.pkl', 'wb') as fp:
    pickle.dump(model_enc, fp)

In [43]:
with open('./Models/train_400_120_dec.pkl', 'wb') as fp:
    pickle.dump(model_dec, fp)

In [44]:
with open('./Models/train_400_120_inp_emb.pkl', 'wb') as fp:
    pickle.dump(model_input_emb, fp)

In [45]:
with open('./Models/train_400_120_out_emb.pkl', 'wb') as fp:
    pickle.dump(model_output_emb, fp)

## Evaluation Step

**Entended Vocabulary**

In [29]:
ext_vocab_indexer = copy.deepcopy(vocab_indexer)
for (x, y) in dev:
    for x_tok, y_tok in zip(tokenize(x), tokenize(y)):
        ext_vocab_indexer.get_index(x_tok)
        ext_vocab_indexer.get_index(y_tok)
        
ext_dev_data_indexed = index_data(dev, ext_vocab_indexer)
ext_dev_data_indexed.sort(key=lambda ex: len(ex.x_indexed), reverse=True)
ext_input_dev_max_len = np.max(np.asarray([len(ex.x_indexed) for ex in ext_dev_data_indexed]))
ext_all_dev_input_data = make_padded_input_tensor(ext_dev_data_indexed, ext_vocab_indexer, ext_input_dev_max_len).astype(np.int64)
ext_X_tensors_batch_dev = batch_data(ext_all_dev_input_data, BATCH_SIZE, cuda=CUDA)   # batch_num, batch_size, sent_len

**Evaluation Copy**

In [30]:
def pointer_generate_dev(p_gen, dec_output, dec_attn, ext_X_tensors, vocab_indexer, ext_vocab_indexer, cuda):
    final_distrib = torch.zeros([dec_output.shape[0], len(ext_vocab_indexer)], dtype=torch.float)    # add new words in the end
    if cuda:
        final_distrib = final_distrib.cuda()
    final_distrib[:, 0:len(vocab_indexer)] = p_gen * dec_output
    dec_attn_padding = torch.zeros(ext_X_tensors.shape, dtype=torch.float)
    if cuda:
        dec_attn_padding = dec_attn_padding.cuda()
    dec_attn_padding[:, 0:dec_attn.shape[1]] = dec_attn
    final_distrib = final_distrib.scatter_add(1, ext_X_tensors, (1-p_gen)*dec_attn_padding)
    final_distrib[:, vocab_indexer.index_of(UNK_SYMBOL)] = 0
    return final_distrib

**Performance Evaluation**

In [57]:
def getModelPerf(scores_lst):
    result_dic = {'rouge-1': {'f':0, 'p':0, 'r':0}, 'rouge-2': {'f':0, 'p':0, 'r':0}, 'rouge-l': {'f':0, 'p':0, 'r':0}}
    for scores in scores_lst:
        for key1 in scores:
            for key2 in scores[key1]:
                result_dic[key1][key2] += scores[key1][key2]
    sent_nums = len(scores_lst)
    for key1 in result_dic:
        for key2 in result_dic[key1]:
            result_dic[key1][key2] /= sent_nums
    return result_dic

**Create indexed input/output for development**

In [31]:
# Create indexed input/output for dev
dev_data_indexed.sort(key=lambda ex: len(ex.x_indexed), reverse=True)
input_dev_max_len = np.max(np.asarray([len(ex.x_indexed) for ex in dev_data_indexed]))
all_dev_input_data = make_padded_input_tensor(dev_data_indexed, vocab_indexer, input_dev_max_len).astype(np.int64)
output_dev_max_len = np.max(np.asarray([len(ex.y_indexed) for ex in dev_data_indexed]))
X_tensors_batch_dev = batch_data(all_dev_input_data, BATCH_SIZE, cuda=CUDA)   # batch_num, batch_size, sent_len
if CUDA:
    inp_lens_batch_dev = [torch.tensor([torch.sum(X_tensor != 0) for X_tensor in X_tensors]).cuda() for X_tensors in X_tensors_batch_dev]  # batch_num, batch_size
else:
    inp_lens_batch_dev = [torch.tensor([torch.sum(X_tensor != 0) for X_tensor in X_tensors]) for X_tensors in X_tensors_batch_dev]  # batch_num, batch_size

**Evaluation**

In [40]:
best_data = []
model_enc.eval()
model_dec.eval()
for X_tensors, inp_lens_tensor, ext_X_tensors in zip(X_tensors_batch_dev, inp_lens_batch_dev, ext_X_tensors_batch_dev):
    enc_outputs, enc_context_mask, enc_hidden = encode_input_for_decoder(X_tensors, inp_lens_tensor, model_input_emb, model_enc)
    dec_hidden = enc_hidden
    init_dec_inp = Variable(torch.LongTensor([vocab_indexer.index_of(SOS_SYMBOL)] * X_tensors.shape[0]))  
    if CUDA:
        init_dec_inp = init_dec_inp.cuda()
    dec_input = model_output_emb.forward(init_dec_inp)
    cont = torch.zeros((X_tensors.shape[0], 2 * hidden_size))    # batch_size, 2*hidden_size
    coverage = torch.zeros((X_tensors.shape[0], enc_outputs.shape[0]))             # batch_size, sent_lens
    all_dec_outputs = Variable(torch.zeros(output_dev_max_len, X_tensors.shape[0], len(ext_vocab_indexer)))   # sent_len, batch_size, ext_output_size
    
    if CUDA:
        cont = cont.cuda()
        coverage = coverage.cuda()
        all_dec_outputs = all_dec_outputs.cuda()
    
    for idx in range(output_dev_max_len):
        p_gen, dec_output, dec_hidden, dec_attn, cont, next_coverage = model_dec.forward(dec_input, dec_hidden, enc_outputs, enc_context_mask, cont, coverage)
        all_dec_outputs[idx] = pointer_generate_dev(p_gen, dec_output, dec_attn, ext_X_tensors, vocab_indexer, ext_vocab_indexer, CUDA)
        max_prob_idx = torch.argmax(all_dec_outputs[idx], dim=1)
        max_prob_idx[max_prob_idx >= len(vocab_indexer)] = vocab_indexer.index_of(UNK_SYMBOL)   # new words should be UNK when serving as next input
        dec_input = model_output_emb.forward(max_prob_idx)
    for best_sent in torch.argmax(all_dec_outputs, dim=2).transpose(0, 1).contiguous():
        best_ex = []
        for word_idx in best_sent:            # don't need to include EOS tok
            if word_idx.item() == ext_vocab_indexer.index_of(EOS_SYMBOL):
                break
            best_ex.append(ext_vocab_indexer.get_object(word_idx.item()))     # pred tok
        best_data.append(best_ex)

Both Mr Brown and Mr Blair rose to prominence when Lord Kinnock led Labour between 1983 and 1992.Former Labour leader Lord Kinnock said the chancellor would be best placed to take over from Mr Blair.Tony Blair has become the Labour Party 's longest-serving prime minister.Labour won a huge majority of 167 over the Conservatives in 2001 , but Mr Blair has since been criticised by many in his own party.Gordon Brown , chancellor of the exchequer under Mr Blair , became Britain 's longest-serving chancellor of modern times in 2004.In 1997 , Mr Blair became the youngest premier of the 20th century , when he came to power at the age of 43 .
Both Mr Brown and Blair Mr Blair rose to prominence when when Labour Labour Labour Labour and Former leader 's 's said take majority since Brown Brown Labour , of And Blair , chancellor chancellor of the the 20th under placed , of May , the and '' last said under would would Blair a future premier by Kinnock system and contest chancellor 's huge Kinnock Ki

BBC correspondent Stephen Cape said the combined unions represented `` a formidable force '' which could embarrass the government in the run-up to the General Election.The UK 's biggest civil service union is to ballot its 290,000 members on strikes in protest at government plans to extend their pension age to 65.The government says unions will be consulted before any changes are made to the pension system.The Public and Commercial Services Union will co-ordinate any action with up to six other public sector unions.PCS leader Mark Serwotka warned there could be further walkouts unless there was a government rethink.Unions have already earmarked 23 March for a one-day strike which could involve up to 1.4 million UK workers .
BBC Stephen Stephen in the which . `` correspondent Stephen , correspondent formidable which correspondent the some the represented General General General General General consulted General the on one pension the pension says up to plans its years which years stoppa

to to Criminal can can public disappointed the confirm found found action case to case a case to the question guilty.Greek of Citigroup the prosecutor has been criticism by the prosecutor move has been disappointed 's the a investigation in evidence Citigroup , '' Citigroup The The a referred to individuals a of , to case the be the the the back 's a investigation said financial German August We should German Citigroup signs of the prosecutor saw Citigroup , brought in bought .
[{'rouge-1': {'f': 0.5245901593697931, 'p': 0.7441860465116279, 'r': 0.4050632911392405}, 'rouge-2': {'f': 0.0846560798689849, 'p': 0.10666666666666667, 'r': 0.07017543859649122}, 'rouge-l': {'f': 0.268467511780758, 'p': 0.4418604651162791, 'r': 0.24050632911392406}}]
Land owners claim the National Land Institute has made mistakes in classifying lands as public or private.Under a 2001 land law , the government can tax or seize unused farm sites.In a statement , Mr Rangel said the land reform is not against the c

Another potential option , Glasgow flanker Andrew Wilson , has been ruled out for a month after damaging ligaments in both knees against Northampton recently.The Borders flanker has a knee injury and joins Donnie Macfadyen and Allister Hogg on the sidelines.Scotland doctor James Robson said : `` A scan has shown damage to the medial ligaments of Scott 's right knee ruling him out of the first part of the Championship.Star number eight Simon Taylor will miss at least the first two games after damaging an ankle during his comeback.Scotland 's back row crisis has worsened ahead of the RBS Six Nations with news that Scott Gray will miss out on the opening matches .
may potential , Another may miss out , ruled , , option to first Simon a knee two Hogg the two two eight Glasgow Simon a . weekend for damaging his and miss Allister opener Robson number for his of damaging Robson damage for the first option , to the prop cartilage in damaging the two two two have Another Another how Gray , Will

[{'rouge-1': {'f': 0.6239999950540799, 'p': 0.6964285714285714, 'r': 0.5652173913043478}, 'rouge-2': {'f': 0.17910447262691534, 'p': 0.16981132075471697, 'r': 0.18947368421052632}, 'rouge-l': {'f': 0.5326020332256153, 'p': 0.6071428571428571, 'r': 0.4927536231884058}}]
Rapper 50 Cent has become the first solo artist to have three singles in the US top five in the same week.Newcomer The Game 's debut reached the top of the charts five weeks ago , while 50 Cent 's second collection The Masscre was released in the US at the end of last week.Last May , R & B star Usher scored a chart first , with three concurrent singles in the US Top 10 , a feat which was previously matched only by The Bee Gees and The Beatles.50 Cent also appears on rap protege The Game 's song How We do , number four in the US but now outside the UK top ten .
Rapper 50 Cent Cent become has become weeks Rapper has become the become remained has has become the 50 while solo in the singles and singles three in the same fiv

[{'rouge-1': {'f': 0.5333333290055556, 'p': 0.8421052631578947, 'r': 0.3902439024390244}, 'rouge-2': {'f': 0.159090904432464, 'p': 0.2153846153846154, 'r': 0.12612612612612611}, 'rouge-l': {'f': 0.33683029823181404, 'p': 0.6578947368421053, 'r': 0.3048780487804878}}]
Ray has been nominated in six Oscar categories including best film and best actor for Jamie Foxx.Oscar-nominated film biopic Ray has surpassed its US box office takings with a combined tally of $ 80m ( £43m ) from DVD and video sales and rentals.Ray director Taylor Hackford , responsible for the classic 1982 film An Officer and a Gentleman , has also received an Oscar nomination in the best director category.In its first week on home entertainment release the film was the number one selling DVD , with the limited edition version coming in at number 11 .
Ray has been nominated its Oscar its Oscar-nominated its its Oscar its its its Oscar for Jamie surpassed and surpassed best ( its its its US with takings £43m surpassed and

**Performance**

In [54]:
rouge = Rouge()
scores_lst = []
for test_ex, best_ex in zip(dev_data_indexed, best_data):
    test_str = ' '.join(test_ex.y_tok)
    best_str = ' '.join(best_ex)
    print(test_str)
    print(best_str)
    scores = rouge.get_scores(best_str, test_str)
    print(scores)
    scores_lst.append(scores[0])

Both Mr Brown and Mr Blair rose to prominence when Lord Kinnock led Labour between 1983 and 1992.Former Labour leader Lord Kinnock said the chancellor would be best placed to take over from Mr Blair.Tony Blair has become the Labour Party 's longest-serving prime minister.Labour won a huge majority of 167 over the Conservatives in 2001 , but Mr Blair has since been criticised by many in his own party.Gordon Brown , chancellor of the exchequer under Mr Blair , became Britain 's longest-serving chancellor of modern times in 2004.In 1997 , Mr Blair became the youngest premier of the 20th century , when he came to power at the age of 43 .
Both Mr Brown and Blair Mr Blair rose to prominence when when Labour Labour Labour Labour and Former leader 's 's said take majority since Brown Brown Labour , of And Blair , chancellor chancellor of the the 20th under placed , of May , the and '' last said under would would Blair a future premier by Kinnock system and contest chancellor 's huge Kinnock Ki

But officials job of but said of are are job in to unemployment was of job job crowd.Holmes Germany , losses million million and effect job of the , top of Schroeder job creation does put to has job job job at job stifling , of labour Gerhard of 0.1 job predicted.Local creation of are labour , 10.8 work .
[{'rouge-1': {'f': 0.44642856700414546, 'p': 0.6756756756756757, 'r': 0.3333333333333333}, 'rouge-2': {'f': 0.07228915208012802, 'p': 0.10344827586206896, 'r': 0.05555555555555555}, 'rouge-l': {'f': 0.23681982866631537, 'p': 0.43243243243243246, 'r': 0.21333333333333335}}]
Excluding the car sector , US retail sales were up 0.6 % in January , twice what some analysts had been expecting.Excluding the car sector , sales rose by just 0.3 % .In December , overall retail sales rose by 1.1 % .US retail sales fell 0.3 % in January , the biggest monthly decline since last August , driven down by a heavy fall in car sales.The 3.3 % fall in car sales had been expected , coming after December 's 

India 's government has given its backing to cheaper and more accessible air travel.When it was set up the firm offered tickets that were 50 % cheaper than other Indian airlines.Air Deccan has ordered 30 Airbus A320 planes in a $ 1.8bn ( £931m ) deal as India 's first low-cost airline expands in the fast-growing domestic market.Beer magnate Vijay Mallya recently set up Kingfisher Airlines , while UK entrepreneur Richard Branson has said he is keen to start a local operation.Air Deccan was set up last year and wants to lure travellers away from the railway network and pricier rivals.The potential of the Indian market has attracted attention at home and abroad .
India 's more has given its backing to cheaper and more accessible accessible travel.When it was set up the firm offered in a $ 1.8bn entrepreneur in in January $ $ 50 offered Deccan it tickets tickets the firm offered it % to expected recently the it accessible was expands it was magnate entrepreneur it said expands was by to by

On Tuesday , Morientes had said : `` I like Liverpool and I am pleased that a club of their stature want to buy me.I have told Madrid that I want it to happen.But Benitez could yet turn his attentions to the younger Anelka should Morientes be reluctant to pledge his future to Liverpool.Newcastle have joined the race to sign Real Madrid striker Fernando Morientes and scupper Liverpool 's bid to snap up the player , according to reports.Real are believed to still want £7m before selling Morientes.If Madrid do not want me then it 's in the best interests of everyone that they are realistic .
On Tuesday , Morientes had said : `` `` I like to : I I want reluctant to should should want has should I want I I want I I I being bidding I could . `` like being being bidding attentions bidding want to stature I . `` like like then attentions had had being being . `` being like to permanent I I played could could . `` have like being being bidding had had had could like to I Madrid turn stature sta

[{'rouge-1': {'f': 0.5233644815791773, 'p': 0.8, 'r': 0.3888888888888889}, 'rouge-2': {'f': 0.2206896505721761, 'p': 0.3076923076923077, 'r': 0.17204301075268819}, 'rouge-l': {'f': 0.3542390110617143, 'p': 0.6571428571428571, 'r': 0.3194444444444444}}]
The Duchy of Lancaster provides the Queen 's private income , while the Duchy of Cornwall provides Prince Charles ' annual income.Aides from the Duchy of Lancaster and Duchy of Cornwall will appear before the Commons Public Accounts Committee.Duchy officials , who will appear before the committee on Monday , are only responsible for generating money.Senior officials at the two bodies generating private income for the Queen and Prince of Wales are to be questioned by MPs.The prince has voluntarily paid income tax - currently 40 % - since 1993 .
The Duchy of Lancaster provides the the Charles . `` `` Duchy appear Wales and of committee two Duchy Charles , while appear appear appear Wales last year 's William William William , while , befor

In [58]:
result_dic = getModelPerf(scores_lst)
print(result_dic)

{'rouge-1': {'f': 0.5672983961105551, 'p': 0.8445771616593384, 'r': 0.44006148343795387}, 'rouge-2': {'f': 0.20048847585455193, 'p': 0.2519793439842693, 'r': 0.17646345408604294}, 'rouge-l': {'f': 0.3980085871232608, 'p': 0.6954182204560879, 'r': 0.3623109768543346}}


In [38]:
dev

[["Byrds producer Melcher dies at 62  Record producer Terry Melcher , who was behind hits by the Byrds , Ry Cooder and the Beach Boys , has died aged 62 .  The son of actress Doris Day , he helped write Kokomo for the Beach Boys , which was used in the movie Cocktail , earning a 1988 Golden Globe nomination . He also produced Mr Tambourine Man for the Byrds , as well as other his such as Turn , Turn Turn . Melcher died on Friday night at his home in Beverly Hills , California , after a long battle with skin cancer . He joined Columbia Records as a producer in the mid-1960s , and also worked with Gram Parsons and the Mamas and the Papas .  Earlier in his career , Melcher had hits as part of duo called Bruce & Terry , with future Beach Boy Bruce Johnston , which evolved into the Rip Chords group . Melcher also worked closely with his mother , producing The Doris Day Show and helping to run her charitable activities . In 1969 his name became linked with the Charles Manson murders , which 

In [39]:
for test_ex, best_ex in zip(dev_data_indexed, best_data):
    test_str = ' '.join(test_ex.y_tok)
    print(len(test_ex.y_tok), test_str)

114 Both Mr Brown and Mr Blair rose to prominence when Lord Kinnock led Labour between 1983 and 1992.Former Labour leader Lord Kinnock said the chancellor would be best placed to take over from Mr Blair.Tony Blair has become the Labour Party 's longest-serving prime minister.Labour won a huge majority of 167 over the Conservatives in 2001 , but Mr Blair has since been criticised by many in his own party.Gordon Brown , chancellor of the exchequer under Mr Blair , became Britain 's longest-serving chancellor of modern times in 2004.In 1997 , Mr Blair became the youngest premier of the 20th century , when he came to power at the age of 43 .
101 More than five million households in the UK have broadband and that number is growing fast.The Demos report looked at the impact of broadband on people 's net habits.More significantly , argues the report , broadband is encouraging people to take a more active role online.The Demos report , entitled Broadband Britain : The End Of Asymmetry ? , was 

70 After their 3-2 win over Manchester City , McClaren said : `` We are playing exciting football , it 's a magnificent result to keep us in the top five.McClaren also praised winger Stewart Downing and strikers Jimmy Floyd Hasselbaink and Mark Viduka , who both ended barren runs in front of goal.He added : `` If Stewart keeps playing like this Sven-Goran Eriksson has got to pick him .
69 Munster 's Heineken Cup quarter-final tie against Biarritz on 3 April has been switched to Real Sociedad 's Paseo de Anoeta stadium in San Sebastian.Ulster were the last Irish team to play at the Paseo de Anoeta stadium where they faced a Euskarians side during a pre-season tour in 1998.Real 's ground holds 32,000 whereas the Parc des Sports Aguilera in Biarritz has a capacity of just 12,667 .
75 Ronaldo refused to commit his long-term future to the club.I 'm happy but nobody knows the future . `` `` The United board have already made an offer to renew the contract but I 'm trying not to think about i

In [ ]:
# Create indexed input/output for training
# train_data_indexed.sort(key=lambda ex: len(ex.x_indexed), reverse=True)
# input_train_max_len = np.max(np.asarray([len(ex.x_indexed) for ex in train_data_indexed]))
# all_train_input_data = make_padded_input_tensor(train_data, input_indexer, input_train_max_len, args.reverse_input)
# output_train_max_len = np.max(np.asarray([len(ex.y_indexed) for ex in train_data]))
# all_train_output_data = make_padded_output_tensor(train_data, output_indexer, output_train_max_len)


# # Create indexed input/output for dev
# dev_data_indexed.sort(key=lambda ex: len(ex.x_indexed), reverse=True)
# input_dev_max_len = np.max(np.asarray([len(ex.x_indexed) for ex in dev_data_indexed]))
# all_dev_input_data = make_padded_input_tensor(dev_data_indexed, input_indexer, input_dev_max_len, args.reverse_input)
# output_dev_max_len = np.max(np.asarray([len(ex.y_indexed) for ex in dev_data_indexed]))
# all_dev_output_data = make_padded_output_tensor(dev_data_indexed, output_indexer, output_dev_max_len)



#         else:
#             for idx in range(output_train_max_len):
# #                 dec_output, dec_hidden = model_dec.forward(dec_input, dec_hidden)
#                 p_gen, dec_output, dec_hidden, dec_attn = model_dec.forward(dec_input, dec_hidden, enc_outputs, enc_context_mask)
#                 all_dec_outputs[idx] = pointer_generate_train(p_gen, dec_output, dec_attn, X_tensors)
#                 max_prob_idx = torch.argmax(all_dec_outputs[idx], dim=1)
#                 dec_input = model_output_emb.forward(max_prob_idx)

#         all_context_mask = torch.from_numpy(np.asarray([[1 if j < oup_lens_tensor.data[i].item() \
#             else 0 for j in range(0, Y_tensors.size(1))] for i in range(0, oup_lens_tensor.shape[0])], dtype=np.uint8))
#         loss = masked_cross_entropy(all_dec_outputs.transpose(0, 1).contiguous(), Y_tensors, oup_lens_tensor, all_context_mask)       # batch_size, sent_len, output_size
#                                                                                                                                       # batch_size, sent_len
#         Y_resize = torch.transpose(Y_tensors, 0, 1).contiguous()
#         loss = criterion(all_dec_outputs.view(-1, all_dec_outputs.shape[2]), Y_resize.view(-1))

# use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False